In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install pycaret[full]

In [ ]:
pip install shutup

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import shutup
shutup.please()

In [2]:
#Training Data

train_data = pd.read_csv('https://raw.githubusercontent.com/Fluid-AI/marketprophecy/main/NSE%20Data/NSE%20Training%20Data%20-%201st%20Jan%202016%20to%201st%20Jan%202022.csv')
train_data.head()

In [3]:
#target variable is 'Adj Close'. We will drop 'Close' column from train data
train_data_1 = train_data.drop('Close', 1)
train_data_1.head()

In [4]:
train_data_1.info()

In [5]:
train_data_1['Date'] = pd.to_datetime(train_data_1['Date'])

In [6]:
train_data_1.info()

In [7]:
train_data_1['year'] = [i.year for i in train_data_1['Date']]
train_data_1['month'] = [i.month for i in train_data_1['Date']]
train_data_1['day'] = [i.dayofweek for i in train_data_1['Date']]
train_data_1.head()

In [8]:
train_data_1.info()

In [9]:
train_data_1.drop('Date',1, inplace=True)
train_data_1.head()

In [10]:
#out of time testing data

test_data = pd.read_csv('https://raw.githubusercontent.com/Fluid-AI/marketprophecy/main/NSE%20Data/NSE%20Out%20of%20Time%20Testing%20Data%20-%201st%20Jan%202022%20to%204th%20Feb%202022.csv')
test_data.head()

In [11]:
test_data.shape

In [12]:
test_data.info()

In [13]:
test_data_1 = test_data.drop(columns=['Close', 'Adj Close'], axis=1)

In [14]:
test_data_1.head()

In [15]:
#For calculating the final error from predicted values and actual values for test data we will seperate target values from test data

y_actual = test_data['Adj Close']

In [16]:
test_data_1['Date'] = pd.to_datetime(test_data_1['Date'])

In [17]:
#we will extract same date features from test data 
test_data_1['year'] = [i.year for i in test_data_1['Date']]
test_data_1['month'] = [i.month for i in test_data_1['Date']]
test_data_1['day'] = [i.dayofweek for i in test_data_1['Date']]
test_data_1.head()

In [18]:
test_data_1.drop('Date', 1, inplace=True)

In [19]:
test_data_1.head()

In [20]:
test_data_1.isnull().sum()

In [21]:
train_data_1.shape

In [22]:
train_data_1.isnull().sum()

In [23]:
#there seems to be six null values. Since it is just six we will drop those six rows
train_data_1.dropna(inplace=True)

In [24]:
train_data_1.isnull().sum()

In [25]:
train_data_1.head()

In [26]:
#We will pycaret timeseries library to create a model
from pycaret.regression import *

In [27]:
#we will setup the training environment using setup function

exp = setup(data = train_data_1,
            target = 'Adj Close',
            normalize = True,
            polynomial_features = True,
            feature_selection = True,
            fold_strategy = 'timeseries',
            fold = 10,
            use_gpu = True,
            log_experiment = True,
            experiment_name = 'exp1',
            log_plots = True,
            profile = True
           )

In [28]:
best_model = compare_models()

#### Lets create a huber regressor to check how it performs on unseen data

In [29]:
model_1 = create_model('huber')

In [30]:
evaluate_model(model_1)

In [31]:
#we will predict values for unseen data

pred_data = predict_model(model_1, data = test_data_1)

In [32]:
pred_data.head()

In [33]:
predicted_values = list(pred_data['Label'])

In [34]:
actual_values = list(y_actual)

In [35]:
df = pd.DataFrame({'Date':list(test_data['Date']), 'actual_values':actual_values, 'predicted_values':predicted_values})

In [36]:
df.head()

In [38]:
#Plot to see Actual vs Predicted values

df.plot()

In [37]:
from sklearn.metrics import r2_score
r2_score(actual_values, predicted_values)

#### We can clearly see that there is overfitting as test score and validation score differ by huge margin

#### Let's create a XGBoost model and see how it performs on unseen data

In [50]:
model_2 = create_model('xgboost')

In [51]:
evaluate_model(model_2)

In [52]:
pred_data_2 = predict_model(model_2, data = test_data_1)

In [53]:
pred_data_2.head()

In [54]:
predicted_values_2 = list(pred_data_2['Label'])

In [55]:
df_2 = pd.DataFrame({'Date':list(test_data['Date']), 'actual_values':actual_values, 'predicted_values':predicted_values_2})
df_2.head()

In [56]:
df_2.plot()

In [57]:
r2_score(actual_values, predicted_values_2)

In [71]:
final_model_2 = finalize_model(model_2)

In [78]:
evaluate_model(final_model_2)

In [72]:
pred_data_2_final = predict_model(final_model_2, data = test_data_1)

In [73]:
pred_data_2_final.head()

In [74]:
predicted_values_2_final = list(pred_data_2_final['Label'])

In [75]:
df_2_final = pd.DataFrame({'Date':list(test_data['Date']), 'actual_values':actual_values, 'predicted_values':predicted_values_2_final})
df_2_final.head()

In [76]:
df_2_final.plot()

In [77]:
r2_score(actual_values, predicted_values_2_final)

#### XGBoost model seems to be better than previous model interms of fitting the data. We can see a val score of 99.75% with train data and test score of 93.8% with out of time test data.

#### Also looking at the learning curve, we need more data to reduce the complexity of the model and increase generalization.